In [ ]:
import os
import time

from efficientnet_pytorch import EfficientNet
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torchvision import datasets, transforms

In [ ]:
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_size = 224
model_path = './models/efficientnet-b3_epoch100.pth'

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=0) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

model_test = EfficientNet.from_pretrained('efficientnet-b3')
num_ftrs = model_test._fc.in_features

model_test._fc  = nn.Linear(num_ftrs, len(class_names))

model_test.to(device)

model_test.load_state_dict(torch.load(model_path))
model_test.eval()
print()

In [ ]:
def inference_model(model, input_image_path):
    was_training = model.training
    model.eval()

    with torch.no_grad():
        with Image.open(input_image_path) as input_image:
            resized_image = data_transforms['val'](input_image).unsqueeze(0).to(device)
            label = input_image_path.split('_')[-1].split('.')[0]
            
            output = model(resized_image)
            _, pred = torch.max(output, 1)
            
        model.train(mode=was_training)
    
    return class_names[pred[0]], label

In [ ]:
fig = plt.figure()

since = time.time()

words = ['2530_田.jpg', '4531_佳.jpg', '62947_科.jpg', '9997_王.jpg', '54240_允.jpg']

for i, word in enumerate(words):
    start = time.time()
    pred, label = inference_model(model_test, os.path.join('./data/clean/', word))
    print(f'Image {i + 1}\nPredict: {pred}, Label: {label}, Time elapsed: {time.time() - start}\n')
    img = mpimg.imread(os.path.join('./data/clean', word))
    plt.imshow(img)
    plt.show()
    
time_elapsed = time.time() - since

print(f'Inference time of one image: {time_elapsed / len(words)}')

In [ ]:
val_dir = './data/val/isnull'
val_words_file = []

for root, dirs, files in os.walk(val_wordset_dir, topdown=True):
    for name in files:
        val_words_file.append(name)

preds = []
labels = []

for i, word in enumerate(val_words):
    pred, label = inference_model(model_test, os.path.join('./data/clean/', word))
    preds.append(pred)
    labels.append('isnull')

print(f'The accuracy of isnull is {accuracy_score(preds, labels)}')

In [ ]:
val_dir = './data/val'
val_words = ['仙', '圖', '迅', '鑽', '勁', '班', '央', '衣', '買', '首', '匠', '厚']
val_words_file = []

for val_word in val_words:
    for root, dirs, files in os.walk(os.path.join(val_dir, val_word), topdown=True):
        for name in files:
            val_words_file.append(name)

preds = []
labels = []

for i, word in enumerate(val_words_file):
    pred, label = inference_model(model_test, os.path.join('./data/clean/', word))
    preds.append(pred)
    labels.append(label)
    
print(f'The accuracy of little data images is {accuracy_score(preds, labels)}')